In [1]:
import os 
import re
import os
from pathlib import Path

In [9]:
with open('unicode_test.txt') as f:
    content = f.read()

In [10]:
content

'\\x84'

In [2]:
# Pattern test for punctuation removal
pattern = re.compile('[%s]' % re.escape('!"#$%&\'()*+,./:;<=>?@[\\]^_`{|}~„«'))

In [3]:
pattern

re.compile(r'[\!\"\#\$\%\&\\'\(\)\*\+\,\.\/\:\;\<\=\>\?\@\[\\\]\^_\`\{\|\}\~\„\«]',
re.UNICODE)

In [137]:
re.sub(r'\d+','\\\za','1Erwerbsunfähigkeit im § 10 Abs 8 a a O kann').strip()

'\\zaErwerbsunfähigkeit im § \\za Abs \\za a a O kann'

##### Test roman numerals remover

In [92]:
test_list = ['wird bei einfachem space hoffentlich nicht replaced',
             ' ', # fuck matching the empty string because I filter out lines < 1 
             '', # # fuck matching space because I filter out lines < 1
    'III Neue',
'XII 0',
'XII 0 IV',
'hallo IV wo',
             '> 1 numeral in a row: klostermann iv ii xx pole',
'numerals between digits: 0 IV 0',
'XII 0 IV jsdfdsjf',
'0 XX sadasd',
             'V',
             '0 V',
             '0 hallo',
             '0 v hallo',
             '0 v 0 IV hallo',
'0 IX sadasd ix',
'0 XI',
'wenn sie es yy MX yy iv yy xii wo irgendwo wollen',
'XX']

In [167]:
test_string = '0 IX sadasd ix'

In [185]:
def replace_roman_numerals(string):
    # Need to run 4 different regex operations in order to capture all roman numerals

    # for roman numerals that appear in the middle of a line
    string = re.sub(r'\b\sM{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})\s\b',' rom ', string, flags=re.IGNORECASE)
    # for roman numerals that are single element of a line
    string = re.sub(r'^M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})$', 'rom', string, flags=re.IGNORECASE)
    # for roman numerals that appear at the beginning of a line that contains other elements 
    string = re.sub(r'^M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})\s' , 'rom ', string, flags=re.IGNORECASE)
    # for roman numerals that appear at the end of a line that contains other elements 
    string = re.sub(r'\sM{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})$', ' rom', string, flags=re.IGNORECASE)
    return string 

In [192]:
with open('data/processed_corpora_BT/WP_12_processed/121_sents.txt', 'r', encoding='utf-8') as f:
    text = f.readlines()
    text = [line.strip() for line in text]

In [193]:
for line in text:
    temp = replace_roman_numerals(line)
    if temp != line:
        print(line)
        print(temp)
        print('\n')

Die Hilfe ist als Sachleistung zu gewähren d h sie muß so gewährt werden das ist der Regelfall
Die Hilfe ist als Sachleistung zu gewähren rom h sie muß so gewährt werden das ist der Regelfall


D Herr Rechtsanwalt oder Herr Kollege sagen soll — In meinem Gemeinderat in dem ich Politik gelernt habe wäre jeder wegen Befangenheit von der Beratung eines Verfahrens ausgeschlossen in dem er als Anwalt mitwirkt
rom Herr Rechtsanwalt oder Herr Kollege sagen soll — In meinem Gemeinderat in dem ich Politik gelernt habe wäre jeder wegen Befangenheit von der Beratung eines Verfahrens ausgeschlossen in dem er als Anwalt mitwirkt


1995 sollen noch 24 900 Stunden Tiefflug über Deutschland geflogen werden also nach der Definition unter 1 500 Fuß oder 450 m Aber die Luftwaffe will insgesamt nur 15 400 Flugstunden absolvieren davon die Hälfte also 7 700 im Inland und 7 700 im Ausland bzw über See
1995 sollen noch 24 900 Stunden Tiefflug über Deutschland geflogen werden also nach der Definition unter 1 

In [186]:
for t in test_list:
    print(replace_roman_numerals(t))

wird bei einfachem space hoffentlich nicht replaced
rom rom
rom
rom Neue
rom 0
rom 0 rom
hallo rom wo
> 1 numeral in a row: klostermann rom ii rom pole
numerals between digits: 0 rom 0
rom 0 rom jsdfdsjf
0 rom sadasd
rom
0 rom
0 hallo
0 rom hallo
0 rom 0 rom hallo
0 rom sadasd rom
0 rom
wenn sie es yy rom yy rom yy rom wo irgendwo wollen
rom


In [120]:
def reduce_numerical_sequences(string): 
    return re.sub(r'((0|xx)\s?){2,}', '\\2 ', string).strip()

In [26]:
reduce_numerical_sequences('wenn sie xx XII irgendwo wollen')

'wenn sie xx XII irgendwo wollen'

In [88]:
for t in test_list:
    print(re.sub(r'\b\sM{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})\s\b',' xx ', t, flags=re.IGNORECASE))
    # für romans in der Mitte

wird bei einfachem space hoffentlich nicht replaced
III xx Neue
XII \za
XII \za IV
XII \za xx jsdfdsjf
\za xx sadasd
V
\za V
\za hallo
\za xx hallo
\za v \za xx hallo
\za xx sadasd ix
\za XI
XX


In [97]:
for t in test_list:
    print(re.sub(r'\b^M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})$\b', '\\\\rom', t))
    # wenn romans als alleinige Zeile dastehen

wird bei einfachem space hoffentlich nicht replaced
III  Neue
XII \za
XII \za IV
XII \za IV jsdfdsjf
\za XX sadasd
\rom
\za V
\za hallo
\za v hallo
\za v \za IV hallo
\za IX sadasd ix
\za XI
\rom


In [92]:
for t in test_list:
    print(re.sub(r'^M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})\s' , '\\\\rom ', t))
    # für Zeilen, die mit romans anfangen und weitere Wörter enthalten

wird bei einfachem space hoffentlich nicht replaced
\rom  Neue
\rom \za
\rom \za IV
\rom \za IV jsdfdsjf
\za XX sadasd
V
\za V
\za hallo
\za v hallo
\za v \za IV hallo
\za IX sadasd ix
\za XI
XX


In [96]:
for t in test_list:
    print(re.sub(r'\sM{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})$', ' \\\\rom', t, flags=re.IGNORECASE))
    # für Zeilen, die weitere Wörter enthalten und mit romans enden

wird bei einfachem space hoffentlich nicht replaced
III  Neue
XII \za
XII \za \rom
XII \za IV jsdfdsjf
\za XX sadasd
V
\za \rom
\za hallo
\za v hallo
\za v \za IV hallo
\za IX sadasd \rom
\za \rom
XX


In [97]:
def replace_roman_numerals(protocols):
    # Need to run 4 different regex operations in order to capture all roman numerals
    
    # for roman numerals that appear in the middle of a line
    protocols = [re.sub(r'\sM{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})\s',' \\\\rom ', line, flags=re.IGNORECASE)
                 for line in protocols]
    # for roman numerals that are single element of a line
    protocols = [re.sub(r'^M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})$', '\\\\rom', line, flags=re.IGNORECASE)
                 for line in protocols]
    # for roman numerals that appear at the beginning of a line but line contains other elements 
    protocols = [re.sub(r'^M{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})\s' , '\\\\rom ', line, flags=re.IGNORECASE)
                 for line in protocols]
    # for roman numerals that appear at the end of a line but line contains other elements 
    return [re.sub(r'\sM{0,4}(CM|CD|D?C{0,3})(XC|XL|L?X{0,3})(IX|IV|V?I{0,3})$', ' \\\\rom', line, flags=re.IGNORECASE)
            for line in protocols]

In [195]:
def replace_roman_numerals_lowercase(protocols):
    # Need to run 4 different regex operations in order to capture all roman numerals
    
    # for roman numerals that appear in the middle of a line
    protocols = [re.sub(r'\sm{0,4}(cm|cd|d?c{0,3})(xc|xl|l?x{0,3})(ix|iv|v?i{0,3})\s',' \\\\rom ', line) for line in protocols]
    # for roman numerals that are single element of a line
    protocols = [re.sub(r'^m{0,4}(cm|cd|d?c{0,3})(xc|xl|l?x{0,3})(ix|iv|v?i{0,3})$', '\\\\rom', line) for line in protocols]
    # for roman numerals that appear at the beginning of a line but line contains other elements 
    protocols = [re.sub(r'^m{0,4}(cm|cd|d?c{0,3})(xc|xl|l?x{0,3})(ix|iv|v?i{0,3})\s' , '\\\\rom ', line) for line in protocols]
    # for roman numerals that appear at the end of a line but line contains other elements 
    return [re.sub(r'\sm{0,4}(cm|cd|d?c{0,3})(xc|xl|l?x{0,3})(ix|iv|v?i{0,3})$', ' \\\\rom', line) for line in protocols]

In [158]:
# Test the noisy digits removal
print(re.sub(r'\b(?P<quote>[A-Za-zß]+)1(?P<name>[A-Za-zß]*)\b', '\g<quote> \g<name>', 'der Privatangestellten1 ... ... . Entwurf'))
print(re.sub(r'\b(?P<quote>[A-Za-zß]+)1(?P<name>[A-Za-zß]*)\b', '\g<quote>\g<name>', 'eines WeingeseHes1 ... ... .hinter Entwurf'))
print(re.sub(r'\b(?P<quote>[A-Za-zßäöü]+)1(?P<name>[A-Za-zßäöü]*)\b', '\g<quote> \g<name>', 'Beiträge1'))

der Privatangestellten  ... ... . Entwurf
eines WeingeseHes ... ... .hinter Entwurf
Beiträge 


In [139]:
re.sub(r'\d+',' \\\za ','1Hinterbliebenversicherung1der')

' \\za Hinterbliebenversicherung \\za der'

##### Test reduce multiple repititions to 1

In [132]:
# re.sub(r'((\\za|\\rom)\s+){2,}', '\\2 ', '\\rom   \\za    \\rom')
re.sub(r'((\\za|\\rom)\s+){2,}', '\\2 ', '\\rom \\rom \\za \\rom \\rom')

'\\rom \\rom'

##### Test minus and dash sign removal

In [50]:
test_dashes = [
'- \\rom - \\za -',
'\\rom -Ganz',
'- \\za - \\za - \\za - \\za - \\za - \\za - \\za - \\za - \\za -',
'sein Reichstag Aktenstück Nr \\rom Pensions- Hinterbliebenenversicherung der Prioatangestellten \\rom Übersicht',
'Mehr — — Zuruf von der FDP Das ist auch besser',
'Hallo wo  befindet sich denn der Trödelladen?',
'  I bes, dUlrike xoxo  '
]

In [44]:
def remove_dash_and_minus_signs(doc):
    # lone standing dash signs at beginning, middle and end of line
    temp = [re.sub(r'\s(-|—|\s)*\s', ' ', line) for line in doc]
    temp = [re.sub(r'^(-|—|\s)*\s', '', line) for line in temp]
    self = [re.sub(r'\b (-|—|\s)*$', '', line) for line in temp]
    return self 

In [51]:
remove_dash_and_minus_signs(test_dashes)

['\\rom \\za',
 '\\rom -Ganz',
 '\\za \\za \\za \\za \\za \\za \\za \\za \\za',
 'sein Reichstag Aktenstück Nr \\rom Pensions- Hinterbliebenenversicherung der Prioatangestellten \\rom Übersicht',
 'Mehr Zuruf von der FDP Das ist auch besser',
 'Hallo wo befindet sich denn der Trödelladen?',
 'I bes, dUlrike xoxo']

##### Char splitting and German chainwords

In [62]:
import sys
sys.path.append(os.path.join(os.getcwd(),'CharSplit'))

import char_split

In [63]:
regex = "([A-Za-z0-9_äÄöÖüÜß]+)[' '][-]?[' ']?[und|oder|als auch|sowie|&]+[' ']-[' ']?([A-Za-z0-9_äÄöÖüÜß]+)"
#     # brackets with following word: usually belonging together in german: (Wirtschafts-)Informatik, building two words
#     regex.append("['(']{1}([A-Za-z0-9_äÄöÖüÜß]+).[')'](.?\w+)")
#     # list of combined words beloning together (3)
#     regex.append("([A-Za-z0-9_äÄöÖüÜß]+)-[,][' ']?([A-Za-z0-9_äÄöÖüÜß]+)-[' ']?[und|oder|sowie|&|,]+[' ']([A-Za-z0-9_äÄöÖüÜß]+-?([A-Za-z0-9_äÄöÖüÜß]+))")
#     # brackets with following word: usually belonging together in german: lv- oder kvbestandsfuehrungssystem, 
#     # building two words but we have to append the second part of the second word to the first word
#     regex.append("([A-Za-z0-9_äÄöÖüÜß]+)-[' ']?[und|oder|sowie|&]+[' ']([A-Za-z0-9_äÄöÖüÜß]+-?([A-Za-z0-9_äÄöÖüÜß]+))")
#     # Wirtschafts-/Informatik
#     regex.append("([A-Za-z0-9_äÄöÖüÜß]+)-['']?['/','&',',']['']?([A-Za-z0-9_äÄöÖüÜß]+)")

In [64]:
regex

"([A-Za-z0-9_äÄöÖüÜß]+)[' '][-]?[' ']?[und|oder|als auch|sowie|&]+[' ']-[' ']?([A-Za-z0-9_äÄöÖüÜß]+)"

In [75]:
m = re.search(regex,sentence)
if m:
    findings = m.groups()

In [93]:
for c in zip(findings[::2], findings[1::2], range(0,len(findings),2)):
    print(c)
    sentencence = sentence.replace(sentence[m.start(c[-1]):m.end(c[-1])], charSplitting(3,c))

('Reichsausgaben', 'Einnahmen', 0)


AttributeError: 'NoneType' object has no attribute 'start'

In [87]:
for c in zip(findings[::2], findings[1::2], findings[1::2]):
    print(c)

('Reichsausgaben', 'Einnahmen', 'Einnahmen')


In [88]:
m.start(c[-1]):m.end(c[-1])

SyntaxError: illegal target for annotation (<ipython-input-88-7046ef7b341b>, line 1)

In [65]:
m = re.search(regex,'Reichsausgaben und -Einnahmen')

if m:
    findings = m.groups()
    
print(findings)

('Reichsausgaben', 'Einnahmen')


In [66]:
word_1 = (char_split.split_compound('Reichsausgaben')[0][-2])
word_2 = (char_split.split_compound('Reichsausgaben')[0][-1])
chainword = 'und'
word_3 = 'Einnahmen'

print('{} {} {}'.format(word_1+word_2.lower(), chainword, word_1+word_3.lower()))

Reichsausgaben und Reichseinnahmen


In [67]:
print(charSplitting(3, findings))

Reichsausgaben und Reichseinnahmen


In [174]:
sentence = 'Bernd liebt sowohl Apfel-, Reis-, Kiwi-, Pfirsich- als auch Musbrei'

In [179]:
regex_several_words = r"([A-Za-z0-9_äÄöÖüÜß]+[\s]?-[\s]?,[\s]?){1,}([A-Za-z0-9_äÄöÖüÜß]+[\s]?-[\s]?,[\s]?){1,}([A-Za-z0-9_äÄöÖüÜß]+[\s]?-[\s]?,[\s]?){1,}([A-Za-z0-9_äÄöÖüÜß]+[\s]?-[\s]?)[und|oder|als auch|sowie|&|,]+[\s]([A-Za-z0-9_äÄöÖüÜß]+-?([A-Za-z0-9_äÄöÖüÜß]+))"
m = re.search(regex_several_words, sentence)
findings = m.groups(),
for c in zip(findings[::4], findings[1::4], findings[2::4], findings[3::4], range(0,len(findings),4)):
    print(c)
#     sentence = sentence.replace(sentence[m.start(c[-1]):m.end(c[-1])], charSplitting(1,c))
# print(sentence)

In [181]:
findings

(('Apfel-, ', 'Reis-, ', 'Kiwi-, ', 'Pfirsich- ', 'Musbrei', 'i'),)

In [227]:
re.findall(r"([A-Za-z0-9_äÄöÖüÜß]+)[' ']-?[' ']?(?:und|oder|als auch|sowie|&)+[' ']-[' ']?([A-Za-z0-9_äÄöÖüÜß]+)", 'Reichsausgaben und -Einnahmen')

[('Reichsausgaben', 'Einnahmen')]

In [ ]:
Die Gründe dafür sind das -Mark-Bilanzgesetz das die Bilanzkontinuität unterbrochen

In [70]:
re.findall("([A-Za-z0-9_äÄöÖüÜß]+)-['']?['/','&',','][' ']?([A-Za-z0-9_äÄöÖüÜß]+)",  'Wirtschafts-& Landressourcen')

[('Wirtschafts', 'Landressourcen')]

In [96]:
regex[3]

'-'

In [278]:
def charSplitting(i,groups,chainword="und"):
    
    word1 = groups[0].replace(" ","")
    word2 = groups[1].replace(" ","")
    if len(groups) >= 4:
        word3 = str(groups[2]).replace(" ","")
    if len(groups) >= 5:
        word4 = str(groups[3]).replace(" ","")
    if i == 0:
        #print("{}{}".format(groups[0],groups[1]),groups[1])
        return "{}{} {} {}".format(word1,word2.lower(),chainword,word2)
    if i == 1:
        if len(word4)>1:
            splitted = char_split.split_compound(word4)[0][-1].lower()
            return "{}{} {} {}{} {} {}".format(word1,word3.split("-")[1],chainword,word2,word3.split("-")[1],chainword,word3)
        else:
            splitted = char_split.split_compound(word3)[0][-1].lower()
            return "{}{} {} {}{} {} {}".format(word1,splitted,chainword,word2,splitted,chainword,word3)
    if i == 2:
        if len(word3)>1:
            splitted = char_split.split_compound(word3)[0][-1].lower()
            return "{}{} {} {}".format(word1,word2.split("-")[1],chainword,word2)
        else:
            splitted = char_split.split_compound(word2)[0][-1].lower()
            return "{}{} {} {}".format(word1,splitted,chainword,word2)
        
    if i == 3:
        splitted = (char_split.split_compound(word1)[0][-2])
        return "{} {} {}{}".format(word1, chainword, splitted, word2.lower())

In [279]:
def removeGermanChainWords(text):
    regex = []
    # brackets with following word: usually belonging together in german: (Wirtschafts-)Informatik, building two words
    regex.append(r"['(']{1}([A-Za-z0-9_äÄöÖüÜß]+).[')'](.?\w+)")
    # list of combined words beloning together (3)
    regex.append(r"([A-Za-z0-9_äÄöÖüÜß]+)[\s]?-[\s]?([A-Za-z0-9_äÄöÖüÜß]+)-[\s]?[und|oder|als auch|sowie|&|,]+[\s]([A-Za-z0-9_äÄöÖüÜß]+-?([A-Za-z0-9_äÄöÖüÜß]+))")
#     regex.append("([A-Za-z0-9_äÄöÖüÜß]+)-[,][' ']?([A-Za-z0-9_äÄöÖüÜß]+)-[' ']?[und|oder|sowie|&|,]+[' ']([A-Za-z0-9_äÄöÖüÜß]+-?([A-Za-z0-9_äÄöÖüÜß]+))")
    # brackets with following word: usually belonging together in german: lv- oder kvbestandsfuehrungssystem, 
    # building two words but we have to append the second part of the second word to the first word
    regex.append(r"([A-Za-z0-9_äÄöÖüÜß]+)[' ']?-[' ']?(?:und|und dem|und des|oder|als auch|sowie|&)+[' ']([A-Za-z0-9_äÄöÖüÜß]+-?([A-Za-z0-9_äÄöÖüÜß]+))")            
    # Reichsausgaben und -Einnahme(
    regex.append(r"([A-Za-z0-9_äÄöÖüÜß]+)[' ']-?[' ']?(?:und|oder|als auch|sowie|&)+[' ']-[' ']?([A-Za-z0-9_äÄöÖüÜß]+)")
    # Wirtschafts-/Informatik
    regex.append(r"([A-Za-z0-9_äÄöÖüÜß]+)-['']?['/','&',',']['']?([A-Za-z0-9_äÄöÖüÜß]+)")

    sentence = text
    m = re.search(regex[0],sentence)
    if m:
        print('regex 0')
        findings = m.groups()
        for c in zip(findings[::2], findings[1::2], range(0,len(findings),2)):
            sentence = sentence.replace(sentence[m.start(c[-1]):m.end(c[-1])], charSplitting(0,c))
        return sentence


    m = re.search(regex[1],sentence)
    if m:
        print('regex 1')
        findings = m.groups()
        for c in zip(findings[::4], findings[1::4], findings[2::4], findings[3::4], range(0,len(findings),4)):
            sentence = sentence.replace(sentence[m.start(c[-1]):m.end(c[-1])], charSplitting(1,c))
        return sentence
    
    m = re.search(regex[2],sentence)
    if m:
        print('regex 2')
        findings = m.groups()
        for c in zip(findings[::3], findings[1::3], findings[2::3], range(0,len(findings),3)):
            sentence = sentence.replace(sentence[m.start(c[-1]):m.end(c[-1])], charSplitting(2,c))
        return sentence
    
    m = re.search(regex[3],sentence)
    if m:
        print('regex 3')
        findings = m.groups()
        for c in zip(findings[::2], findings[1::2], range(0,len(findings),2)):
            sentence = sentence.replace(sentence[m.start(c[-1]):m.end(c[-1])], charSplitting(3,c))
        return sentence
    
    #m = re.search(regex[4],sentence)
    #if m:
        #findings = m.groups()
        #for c in zip(findings[::2], findings[1::2], range(0,len(findings),2)):
            #sentence = sentence.replace(sentence[m.start(c[-1]):m.end(c[-1])], charsplitting(3,c))
            
    return sentence

In [262]:
regex = "([A-Za-z0-9_äÄöÖüÜß]+)[' ']-?[' ']?(?:und|oder|als auch|sowie|&)+[' ']-[' ']?([A-Za-z0-9_äÄöÖüÜß]+)"

sentence = 'Die Gründe dafür sind zu hohe Reichsausgaben und -Einnahmen.'
m = re.search(regex, sentence)

In [238]:
if m:
    findings = m.groups()
    print(findings)
    for c in zip(findings[::2], findings[1::2], range(0,len(findings),2)):
        sentence = sentence.replace(sentence[m.start(c[-1]):m.end(c[-1])], charSplitting(3,c))
        print(sentence)

('Reichsausgaben', 'Einnahmen')
Die Gründe dafür sind zu hohe Reichsausgaben und Reichseinnahmen.


In [69]:
findings = m.groups()

tuple

In [92]:
for c in zip(findings[::2], findings[1::2], range(0,len(findings),2)):
    print(charSplitting(0,c))

Wirtschaftsinformatik und Informatik


In [87]:
for c in zip(findings[::2], findings[1::2], range(0,len(findings),2)):
    print(c)

('Wirtschafts', 'Informatik', 0)


In [81]:
sentence[m.start(c[-1]):m.end(c[-1])]

'(Wirtschafts-)Informatik'

In [60]:
    if m:
        findings = m.groups()
        for c in zip(findings[::2], findings[1::2], range(0,len(findings),2)):
            sentence = sentence.replace(sentence[m.start(c[-1]):m.end(c[-1])], charSplitting(0,c))

NameError: name 'm' is not defined

In [282]:
test_lines = [
'Im Studium der (Wirtschafts-)Informatik schreiben sich immer mehr Studenten ein.',
'Hier stellt sich die Gretchenfrage der gesamten Wirtschafts- Finanz- und Kreditpolitik.',
'Marvin und Sandy haben in der Löffel- und Hündchenstellung sehr viel Spaß miteinander.',
'Martina hat gegenüber ihrem Rainer in Geld-& Beziehungsfragen die Hosen an.',
'Wir haben nämlich Menschen die keine Lohn- sowie Einkommensteuer zu bezahlen brauchen aber trotzdem bereit sind zu sparen wenn ihnen für dieses Sparen eine gewisse Belohnung gegeben wird'
]

In [283]:
for line in test_lines:
    print(removeGermanChainWords(line))

regex 0
Im Studium der Wirtschaftsinformatik und Informatik schreiben sich immer mehr Studenten ein.
regex 1
Hier stellt sich die Gretchenfrage der gesamten Wirtschaftspolitik und Finanzpolitik und Kreditpolitik.
regex 2
Marvin und Sandy haben in der Löffelstellung und Hündchenstellung sehr viel Spaß miteinander.
regex 2
Martina hat gegenüber ihrem Rainer in Geldfragen und Beziehungsfragen die Hosen an.
regex 2
Wir haben nämlich Menschen die keine Lohnsteuer und Einkommensteuer zu bezahlen brauchen aber trotzdem bereit sind zu sparen wenn ihnen für dieses Sparen eine gewisse Belohnung gegeben wird


In [91]:
test_words = [
'Budget- Kommission',
'Petitions- ',
'Physikalisch-technische',
'Bankgesetz- Kommission',
'Kleist-Retzow',
'Klein -Angestellter',
'Garde-du-Corps-Kaserne',
'die- ',
]

In [8]:
def remove_dash_and_minus_signs(line):
    temp = [re.sub(r'\s[-—]\s', ' ', line)]
    temp = [re.sub(r'[-—]\s', ' ', line)]
    text = [re.sub(r'\s[-—]', ' ', line)]
    return text 

In [9]:
for word in test_words:
    print(remove_dash_and_minus_signs(word))

['Budget- Kommission']
['Petitions- ']
['Physikalisch-technische']
['Bankgesetz- Kommission']
['Kleist-Retzow']
['Klein Angestellter']
['Garde-du-Corps-Kaserne']
['die- ']


In [25]:
def expandCompoundToken(text, split_chars="-"):
    new_text = text
    for t in text.split():
        parts = []
        add = False   # signals if current part should be appended to previous part
        for p in t.split(split_chars):  # for each part p in compound token t
            if not p: continue  # skip empty part
            if add and parts:   # append current part p to previous part
                parts[-1] += p.lower()
            else:               # add p as separate token
                parts.append(p)
            add = len(p) <= 10   # if p only consists of a single character -> append the next p to it
            
        if len(parts)>0:
            new_text = new_text.replace(t, "-".join(parts))
    return new_text

In [29]:
test_text = 'Ich schreibe eine E-Mail an die Budget- Kommission und die Petitions- physikalisch-technische Anstalt der \
Deutschland GmbH mit Hauptsitz Kleist-Retzow, genauer BMUB-Auftrag gesagt in die- Straße wo steht die Garde-du-Corps-Kaserne'

In [30]:
expandCompoundToken(test_text)

'Ich schreibe eine Email an die Budget Kommission und die Petitions physikalisch-technische Anstalt der Deutschland GmbH mit Hauptsitz Kleistretzow, genauer BMUBauftrag gesagt in die Straße wo steht die Gardeducorpskaserne'

In [28]:
expandCompoundToken('Die Einfuhr- und Ausfuhrmaßnahmen schmecken gar nicht')

'Die Einfuhr und Ausfuhrmaßnahmen schmecken gar nicht'

In [11]:
'Budget-'.split('-')

['Budget', '']

### Test methods for extracting the session protocols from the big document

In [1]:
# Method to test extraction method for the last batch
re.findall(r'Die Sitzung wird um 0 Uhr(?:\s0 Minuten){0,1}(?:\sabends){0,1} durch den Präsidenten eröffnet', 
          'Die Sitzung wird um 0 Uhr durch den Präsidenten eröffnet')

NameError: name 're' is not defined

In [66]:
re.findall(r'Sitzung (?:am\s){0,1}(?:Montag|Dienstag|Mittwoch|Donnerstag|Freitag|Sonnabend|Sonntag) den \d+\.? (?:Januar|Februar|März|April|Mai|Juni|Juli|August|September|Oktober|October|November|Dezember) %s'% 1895,
                    'Sitzung Montag den 10 Januar 1895')
    

['Sitzung Montag den 10 Januar 1895']

### How to sort the documents from glob in an appropriate way?

In [261]:
import glob
sorted(glob.glob('../protocols_1/doc_*.txt'), key=os.path.getmtime)
# glob.glob('../protocols_1/doc_*.txt')

['../protocols_1\\doc_1818.txt',
 '../protocols_1\\doc_1819.txt',
 '../protocols_1\\doc_1820.txt',
 '../protocols_1\\doc_1821.txt',
 '../protocols_1\\doc_1822.txt',
 '../protocols_1\\doc_1823.txt',
 '../protocols_1\\doc_1824.txt',
 '../protocols_1\\doc_1825.txt',
 '../protocols_1\\doc_1826.txt',
 '../protocols_1\\doc_1827.txt',
 '../protocols_1\\doc_1828.txt',
 '../protocols_1\\doc_1829.txt',
 '../protocols_1\\doc_1830.txt',
 '../protocols_1\\doc_1831.txt',
 '../protocols_1\\doc_1832.txt',
 '../protocols_1\\doc_1833.txt',
 '../protocols_1\\doc_1834.txt',
 '../protocols_1\\doc_1835.txt',
 '../protocols_1\\doc_1836.txt',
 '../protocols_1\\doc_1837.txt',
 '../protocols_1\\doc_1838.txt',
 '../protocols_1\\doc_1839.txt',
 '../protocols_1\\doc_1840.txt',
 '../protocols_1\\doc_1841.txt',
 '../protocols_1\\doc_1842.txt',
 '../protocols_1\\doc_1843.txt',
 '../protocols_1\\doc_1844.txt',
 '../protocols_1\\doc_1845.txt',
 '../protocols_1\\doc_1846.txt',
 '../protocols_1\\doc_1847.txt',
 '../proto